In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from mmwrapper.src.api import get_runner

In [2]:
runner = get_runner(r"C:\Users\philmarq\source\repos\MMWrapperRepo\mmwrapper\src\configs\object_detection\object_detection.yaml")

08/03 16:06:21 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.9.16 (main, May 17 2023, 17:49:16) [MSC v.1916 64 bit (AMD64)]
    CUDA available: True
    numpy_random_seed: 146737921
    GPU 0: NVIDIA GeForce RTX 3090 Ti
    CUDA_HOME: None
    MSVC: n/a, reason: fileno
    PyTorch: 2.0.1+cu117
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 199711
  - MSVC 193431937
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm

08/03 16:06:30 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
08/03 16:06:30 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

In [ ]:
runner.train()

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer2.0.conv1.weight:lr=1e-05
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer2.0.conv1.weight:weight_decay=0.0001
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer2.0.conv1.weight:lr_mult=0.1
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer2.0.conv2.weight:lr=1e-05
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer2.0.conv2.weight:weight_decay=0.0001
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer2.0.conv2.weight:lr_mult=0.1
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer2.0.conv3.weight:lr=1e-05
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer2.0.conv3.weight:weight_decay=0.0001
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer2.0.conv3.weight:lr_mult=0.1
08/03 16:06

08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer3.3.conv1.weight:lr_mult=0.1
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer3.3.conv2.weight:lr=1e-05
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer3.3.conv2.weight:weight_decay=0.0001
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer3.3.conv2.weight:lr_mult=0.1
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer3.3.conv3.weight:lr=1e-05
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer3.3.conv3.weight:weight_decay=0.0001
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer3.3.conv3.weight:lr_mult=0.1
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer3.4.conv1.weight:lr=1e-05
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer3.4.conv1.weight:weight_decay=0.0001
08/03 16:06:32 - mmengine - INFO - paramwise_options -- backbone.layer3.4.conv1.weight:lr_mult

08/03 16:06:36 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
08/03 16:06:36 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
08/03 16:06:36 - mmengine - INFO - Checkpoints will be saved to C:\Users\philmarq\source\repos\MMWrapperRepo\firstapitest.


C:\Users\philmarq\miniconda3\envs\mmtest\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


08/03 16:06:39 - mmengine - INFO - Exp name: dino_r50_20230803_160620
08/03 16:06:39 - mmengine - INFO - Epoch(train)  [1][1/1]  lr: 1.0000e-04  eta: 0:02:13  time: 2.7271  data_time: 0.0208  memory: 2383  grad_norm: 112.5889  loss: 55.7638  loss_cls: 1.1273  loss_bbox: 2.1077  loss_iou: 1.9419  d0.loss_cls: 0.9092  d0.loss_bbox: 2.1077  d0.loss_iou: 1.9419  d1.loss_cls: 1.0696  d1.loss_bbox: 2.1077  d1.loss_iou: 1.9419  d2.loss_cls: 0.9659  d2.loss_bbox: 2.1077  d2.loss_iou: 1.9419  d3.loss_cls: 1.1375  d3.loss_bbox: 2.1077  d3.loss_iou: 1.9419  d4.loss_cls: 1.0523  d4.loss_bbox: 2.1077  d4.loss_iou: 1.9419  enc_loss_cls: 0.8258  enc_loss_bbox: 2.1077  enc_loss_iou: 1.9419  dn_loss_cls: 1.1532  dn_loss_bbox: 0.9758  dn_loss_iou: 1.1653  d0.dn_loss_cls: 1.3113  d0.dn_loss_bbox: 0.9758  d0.dn_loss_iou: 1.1653  d1.dn_loss_cls: 1.3708  d1.dn_loss_bbox: 0.9758  d1.dn_loss_iou: 1.1653  d2.dn_loss_cls: 1.2626  d2.dn_loss_bbox: 0.9758  d2.dn_loss_iou: 1.1653  d3.dn_loss_cls: 1.2624  d3.dn_los

...

08/03 16:07:37 - mmengine - INFO - Epoch(val) [9][1/1]    coco/bbox_mAP: 0.0000  coco/bbox_mAP_50: 0.0000  coco/bbox_mAP_75: 0.0000  coco/bbox_mAP_s: -1.0000  coco/bbox_mAP_m: 0.0000  coco/bbox_mAP_l: -1.0000  data_time: 0.0156  time: 0.3063
08/03 16:07:37 - mmengine - INFO - Exp name: dino_r50_20230803_160620
08/03 16:07:37 - mmengine - INFO - Epoch(train) [10][1/1]  lr: 9.2216e-05  eta: 0:00:26  time: 0.6653  data_time: 0.0087  memory: 2837  grad_norm: 203.4448  loss: 52.2725  loss_cls: 1.1497  loss_bbox: 2.2907  loss_iou: 1.8638  d0.loss_cls: 0.7237  d0.loss_bbox: 2.3421  d0.loss_iou: 1.8848  d1.loss_cls: 0.7360  d1.loss_bbox: 2.3110  d1.loss_iou: 1.8788  d2.loss_cls: 0.8220  d2.loss_bbox: 2.2919  d2.loss_iou: 1.8787  d3.loss_cls: 0.9415  d3.loss_bbox: 2.3114  d3.loss_iou: 1.8633  d4.loss_cls: 2.2230  d4.loss_bbox: 2.3013  d4.loss_iou: 1.8636  enc_loss_cls: 0.7040  enc_loss_bbox: 2.2731  enc_loss_iou: 1.8963  dn_loss_cls: 0.5993  dn_loss_bbox: 0.9620  dn_loss_iou: 1.1079  d0.dn_loss

08/03 16:07:52 - mmengine - INFO - Epoch(val) [12][1/1]    coco/bbox_mAP: 0.0000  coco/bbox_mAP_50: 0.0000  coco/bbox_mAP_75: 0.0000  coco/bbox_mAP_s: -1.0000  coco/bbox_mAP_m: 0.0000  coco/bbox_mAP_l: -1.0000  data_time: 0.0156  time: 0.3125
08/03 16:07:53 - mmengine - INFO - Exp name: dino_r50_20230803_160620
08/03 16:07:53 - mmengine - INFO - Epoch(train) [13][1/1]  lr: 8.6448e-05  eta: 0:00:22  time: 0.6128  data_time: 0.0086  memory: 2922  grad_norm: 166.3627  loss: 51.9919  loss_cls: 1.0173  loss_bbox: 2.2975  loss_iou: 1.9171  d0.loss_cls: 0.7111  d0.loss_bbox: 2.3627  d0.loss_iou: 1.9311  d1.loss_cls: 0.7055  d1.loss_bbox: 2.3330  d1.loss_iou: 1.9256  d2.loss_cls: 0.7665  d2.loss_bbox: 2.3130  d2.loss_iou: 1.9256  d3.loss_cls: 0.8573  d3.loss_bbox: 2.3235  d3.loss_iou: 1.9147  d4.loss_cls: 1.8442  d4.loss_bbox: 2.3108  d4.loss_iou: 1.9156  enc_loss_cls: 0.7137  enc_loss_bbox: 2.3148  enc_loss_iou: 1.9409  dn_loss_cls: 0.5891  dn_loss_bbox: 0.9750  dn_loss_iou: 1.1405  d0.dn_los

08/03 16:08:09 - mmengine - INFO - Epoch(val) [15][1/1]    coco/bbox_mAP: 0.0000  coco/bbox_mAP_50: 0.0000  coco/bbox_mAP_75: 0.0000  coco/bbox_mAP_s: -1.0000  coco/bbox_mAP_m: 0.0000  coco/bbox_mAP_l: -1.0000  data_time: 0.0156  time: 0.3741
08/03 16:08:09 - mmengine - INFO - Exp name: dino_r50_20230803_160620
08/03 16:08:09 - mmengine - INFO - Epoch(train) [16][1/1]  lr: 7.9389e-05  eta: 0:00:19  time: 0.5780  data_time: 0.0089  memory: 2120  grad_norm: 142.9643  loss: 51.2795  loss_cls: 0.9423  loss_bbox: 2.2604  loss_iou: 1.9164  d0.loss_cls: 0.6879  d0.loss_bbox: 2.3149  d0.loss_iou: 2.0097  d1.loss_cls: 0.6741  d1.loss_bbox: 2.2856  d1.loss_iou: 1.9820  d2.loss_cls: 0.7205  d2.loss_bbox: 2.2933  d2.loss_iou: 1.9227  d3.loss_cls: 0.7956  d3.loss_bbox: 2.2948  d3.loss_iou: 1.9141  d4.loss_cls: 1.5998  d4.loss_bbox: 2.2772  d4.loss_iou: 1.9151  enc_loss_cls: 0.7084  enc_loss_bbox: 2.3116  enc_loss_iou: 1.9368  dn_loss_cls: 0.5491  dn_loss_bbox: 0.9818  dn_loss_iou: 1.1658  d0.dn_los

08/03 16:08:26 - mmengine - INFO - Epoch(val) [18][1/1]    coco/bbox_mAP: 0.0000  coco/bbox_mAP_50: 0.0000  coco/bbox_mAP_75: 0.0000  coco/bbox_mAP_s: -1.0000  coco/bbox_mAP_m: 0.0000  coco/bbox_mAP_l: -1.0000  data_time: 0.0128  time: 0.3413
08/03 16:08:26 - mmengine - INFO - Exp name: dino_r50_20230803_160620
08/03 16:08:26 - mmengine - INFO - Epoch(train) [19][1/1]  lr: 7.1289e-05  eta: 0:00:17  time: 0.5596  data_time: 0.0087  memory: 2663  grad_norm: 128.6223  loss: 50.9070  loss_cls: 0.8669  loss_bbox: 2.2260  loss_iou: 1.9396  d0.loss_cls: 0.6899  d0.loss_bbox: 2.3109  d0.loss_iou: 2.0170  d1.loss_cls: 0.6669  d1.loss_bbox: 2.2676  d1.loss_iou: 1.9915  d2.loss_cls: 0.6969  d2.loss_bbox: 2.2675  d2.loss_iou: 1.9417  d3.loss_cls: 0.7469  d3.loss_bbox: 2.2642  d3.loss_iou: 1.9356  d4.loss_cls: 1.4212  d4.loss_bbox: 2.2449  d4.loss_iou: 1.9375  enc_loss_cls: 0.7016  enc_loss_bbox: 2.3025  enc_loss_iou: 1.9562  dn_loss_cls: 0.5217  dn_loss_bbox: 0.9957  dn_loss_iou: 1.1864  d0.dn_los

08/03 16:08:43 - mmengine - INFO - Epoch(val) [21][1/1]    coco/bbox_mAP: 0.0000  coco/bbox_mAP_50: 0.0000  coco/bbox_mAP_75: 0.0000  coco/bbox_mAP_s: -1.0000  coco/bbox_mAP_m: 0.0000  coco/bbox_mAP_l: -1.0000  data_time: 0.0138  time: 0.3897
08/03 16:08:44 - mmengine - INFO - Exp name: dino_r50_20230803_160620
08/03 16:08:44 - mmengine - INFO - Epoch(train) [22][1/1]  lr: 6.2434e-05  eta: 0:00:15  time: 0.5480  data_time: 0.0091  memory: 2550  grad_norm: 117.0766  loss: 49.5819  loss_cls: 0.8073  loss_bbox: 2.1228  loss_iou: 1.8904  d0.loss_cls: 0.6636  d0.loss_bbox: 2.2161  d0.loss_iou: 1.9763  d1.loss_cls: 0.6386  d1.loss_bbox: 2.1840  d1.loss_iou: 1.9414  d2.loss_cls: 0.6498  d2.loss_bbox: 2.1858  d2.loss_iou: 1.8979  d3.loss_cls: 0.6952  d3.loss_bbox: 2.1730  d3.loss_iou: 1.8904  d4.loss_cls: 1.2884  d4.loss_bbox: 2.1454  d4.loss_iou: 1.8920  enc_loss_cls: 0.6812  enc_loss_bbox: 2.2071  enc_loss_iou: 1.9126  dn_loss_cls: 0.5107  dn_loss_bbox: 1.0079  dn_loss_iou: 1.2066  d0.dn_los

# Instance seg inference

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#import pydensecrf.densecrf as dcrf

from mmdet.apis import init_detector, inference_detector

import numpy as np
import cv2



config_file = r"C:\Users\philmarq\source\repos\MMWrapperRepo\firstapitest\20230725_162809\vis_data\config.py"
checkpoint_file = r"C:\Users\philmarq\source\repos\MMWrapperRepo\firstapitest\best_coco_bbox_mAP_epoch_16.pth"

device = 'cuda:0'

model = init_detector(config_file, checkpoint_file, device=device)

# Load image


colors = [(255,0,0), (125,0,0), (0,255,0), (50, 255,0), (0,25,255), (0, 125, 0)]
root = r"C:\HSA\HSA-Data\projects\d55090aa-a315-4ae3-8242-b50e13e355f6\dataset\val2017"
for file in os.listdir(root):
    img = os.path.join(root, file)
    result = inference_detector(model, img)
    
    

    color = (255,0,0)
    image = cv2.imread(img)
    #image[image>0] = 0
    masks = result.pred_instances.masks.cpu().numpy()
    scores = result.pred_instances.scores.cpu().numpy()
    labels = result.pred_instances.labels.cpu().numpy()
    bboxes = result.pred_instances.bboxes.cpu().numpy()
    thresh = 0.75
    
    for mask, score,label, bbox in zip(masks, scores,  labels, bboxes):
        if score > thresh:
            if label == 0:  # For label == 0, draw contours
                mask = mask.astype(np.uint8)
                contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                #contours = connect_closest_contours(contours, image, angle_tolerance = 5, fixed_threshold=50, pixel_threshold = 3)
                #color  = np.random.rand(3) * 255
                cv2.drawContours(image, contours, -1, color, -1)
            else:  # For other labels, draw bounding boxes
                color = colors[label]
                x1, y1, x2, y2 = bbox.astype(np.int32)
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 1)
    
    cv2.imwrite(os.path.join("outputall", file), image)